In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

/Users/mukulrathi/anaconda/envs/dltestenv/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dataset = pd.read_csv('arrhythmia.data.txt')
dataset = dataset.replace('?', 0)
#dataset.convert_objects(convert_numeric=True)


In [3]:
#randomly shuffle the data 
dataset = dataset.sample(frac=1).reset_index(drop=True)
X = dataset.iloc[:,0:279] 
Y = dataset.iloc[:,279:]


X = X.as_matrix()
Y = Y.as_matrix()
Y-=1
Y = np.squeeze(np.eye(16)[Y])

In [4]:
hyperparameters={}
hyperparameters["num_epochs"] = 100 #number of passes through the training set
hyperparameters["batch_size"] = 128 #number of examples trained upon in each step of training
hyperparameters["learning_rate"] = 1e-3
hyperparameters["n_x"] = 279
hyperparameters["n_y"] = 16
hyperparameters["layers_units"] = [1024, 512, 512,512,256,256,128,32,16]

In [5]:
TrainTestDevSets = {}

TrainTestDevSets["X_train"] = X[:330,:]
TrainTestDevSets["Y_train"] = Y[:330,:]

TrainTestDevSets["X_dev"] = X[330:400,:]
TrainTestDevSets["Y_dev"] = Y[330:400,:]

TrainTestDevSets["X_test"] = X[400:,:]
TrainTestDevSets["Y_test"] = Y[400:,:]

In [6]:
def initialise_parameters(weight_shape, bias_shape):
    weight = tf.Variable(tf.truncated_normal(weight_shape, stddev=0.1))
    bias = tf.Variable(tf.constant(0.1, shape=bias_shape))
    return weight, bias

In [7]:
def fully_connected_layer(X, num_units,layer_num):
    with tf.name_scope('fc'+str(layer_num)):
        input_dim = int(X.shape[1])
        weight_shape = [input_dim,num_units]
        bias_shape = [num_units]
        Weight, bias = initialise_parameters(weight_shape,bias_shape)
        activation = tf.nn.relu(tf.matmul(X, Weight) + bias)
        #tf.summary.histogram("W_fc" + str(layer_num), Weight)
        #tf.summary.histogram("b_fc" + str(layer_num), bias)
        #tf.summary.histogram("Activation_fc"+ str(layer_num), activation)
    return activation

In [8]:
def forward_propagation(X,layers_units):
    self = fully_connected_layer(X,layers_units[0],1)
    for i in range(1,len(layers_units)-1):
        self = fully_connected_layer(self,layers_units[i],i+1)
    with tf.name_scope('fc'+str(len(layers_units))):
        input_dim = int(self.shape[1])
        weight_shape = [input_dim,layers_units[len(layers_units)-1]]
        bias_shape = [layers_units[len(layers_units)-1]]
        Weight, bias = initialise_parameters(weight_shape,bias_shape)
        self = tf.matmul(self, Weight) + bias
    return self

In [9]:
def cost_function(ZL,Y):
    logits = tf.transpose(ZL)
    labels = tf.transpose(Y)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels,logits=logits))
    return cost

In [10]:
def eval_accuracy(ZL,Y):
    correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(ZL,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("Accuracy", accuracy)
    return accuracy

In [11]:
def softmax(x):
    x-= np.max(x,axis=1,keepdims=True)
    x = np.exp(x)
    x/= np.sum(x,axis=1,keepdims=True)
    return x

In [12]:
def train_model(hyperparameters,TrainTestDevSets):
    tf.reset_default_graph()
    
    num_epochs = hyperparameters["num_epochs"]
    batch_size = hyperparameters["batch_size"]
    learning_rate = hyperparameters["learning_rate"]
    layers_units = hyperparameters["layers_units"]

    n_x = hyperparameters["n_x"]
    n_y = hyperparameters["n_y"]
    X_train = TrainTestDevSets["X_train"]
    Y_train = TrainTestDevSets["Y_train"]
    X_dev = TrainTestDevSets["X_dev"]
    Y_dev = TrainTestDevSets["Y_dev"]
    X_test = TrainTestDevSets["X_test"]
    Y_test = TrainTestDevSets["Y_test"]

    
    X = tf.placeholder(tf.float32,[None,n_x])
    Y = tf.placeholder(tf.float32,[None,n_y])
    ZL = forward_propagation(X,layers_units)
       
    cost = cost_function(ZL,Y)
    
    accuracy = eval_accuracy(ZL,Y)
   
    optimiser = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    #summaries = tf.summary.merge_all()
    
    # Initialize all the variables
    init = tf.global_variables_initializer()
    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        sess.run(init)
        #summary_writer = tf.summary.FileWriter("temp/training/",tf.get_default_graph())
        print("Training the model: ")
        for epoch in range (num_epochs):
            for i in range(0,X_train.shape[1]//batch_size):
                #get the next minibatch to train on
                X_train_minibatch = X_train[:,i*batch_size:(i+1)*batch_size]
                Y_train_minibatch = Y_train[:,i*batch_size:(i+1)*batch_size]
            
                sess.run([optimiser], feed_dict={X: X_train, Y: Y_train})
                #summary_writer.add_summary(summary, epoch)
                train_accuracy = accuracy.eval(feed_dict={X: X_train, Y: Y_train})
                print('Epoch '+ str(epoch) +  ': train accuracy = ' + str(train_accuracy))
                dev_accuracy = accuracy.eval(feed_dict={X: X_dev, Y: Y_dev})
                print('Dev accuracy = ' + str(dev_accuracy))
        print("Training complete!")   
        test_accuracy = accuracy.eval(feed_dict={X: X_test, Y: Y_test})
        print('The test accuracy = ' + str(test_accuracy))
        Y_predict = ZL.eval({X:X_test})
        Y_predict = pd.DataFrame(softmax(Y_predict))
        Y_test = pd.DataFrame(Y_test)
        return Y_predict,Y_test
    


In [13]:
Y_predict,Y_test = train_model(hyperparameters,TrainTestDevSets)

Training the model: 
Epoch 0: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 0: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 1: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 1: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 2: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 2: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 3: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 3: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 4: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 4: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 5: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 5: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 6: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 6: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 7: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 7: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 8: train accuracy = 0.527273


Dev accuracy = 0.542857
Epoch 69: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 69: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 70: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 70: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 71: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 71: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 72: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 72: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 73: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 73: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 74: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 74: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 75: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 75: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 76: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 76: train accuracy = 0.527273
Dev accuracy = 0.542857
Epoch 77: train 

In [14]:
Y_predict

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
Y_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
Y_predict.to_csv('NN_predictions',encoding='utf-8',index=False)
Y_test.to_csv('NN_test',encoding='utf-8',index=False)

